Урок 4. Система управления базами данных MongoDB в Python

1. Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:
название источника;
наименование новости;
ссылку на новость;
дата публикации.
2. Сложить собранные новости в БД
Минимум один сайт, максимум - все три

In [3]:
import requests
from pymongo import MongoClient
from lxml import html
from pprint import pprint

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/97.0.4692.99 Safari/537.36'}

client = MongoClient('127.0.0.1', 27017)
db = client['news_db']
lenta_news = db.lenta_news
lenta_news.drop()

url = 'https://lenta.ru/'
response = requests.get(url, headers=headers)

dom = html.fromstring(response.text)
items = dom.xpath('//div/a[contains(@class,"card-") and contains(@class,"_topnews")]')

for item in items:
    site = 'lenta.ru'
    name = item.xpath('.//*[contains(@class,"_title")]/text()')[0]
    link = url + item.get('href')
    date = '.'.join([s for s in item.get('href').split('/') if s.isdigit()])
    
    news = {
        'site': site,
        'name': name,
        'link': link,
        'date': date,
    }

    try:
        lenta_news.insert_one(news)
    except:
        continue

for doc in lenta_news.find({}):
    pprint(doc)

{'_id': ObjectId('63d6aea3aa41a767e558c8ab'),
 'date': '2023.01.29',
 'link': 'https://lenta.ru//news/2023/01/29/ifeelsorry/',
 'name': 'Российские войска взяли под контроль девять населенных пунктов в '
         'Запорожье',
 'site': 'lenta.ru'}
{'_id': ObjectId('63d6aea3aa41a767e558c8ac'),
 'date': '2023.01.29',
 'link': 'https://lenta.ru//news/2023/01/29/vnokovoko/',
 'name': 'Во Внуково полицейские задержали молодую девушку с полкило кокаина в '
         'желудке',
 'site': 'lenta.ru'}
{'_id': ObjectId('63d6aea3aa41a767e558c8ad'),
 'date': '2023.01.29',
 'link': 'https://lenta.ru//news/2023/01/29/iranjadern/',
 'name': 'В США сообщили о рассмотрении вариантов по недопущению Ирана к '
         'ядерному оружию',
 'site': 'lenta.ru'}
{'_id': ObjectId('63d6aea3aa41a767e558c8ae'),
 'date': '2023.01.29',
 'link': 'https://lenta.ru//news/2023/01/29/duhamel/',
 'name': 'Чемпионка ОИ призвала страны бойкотировать Игры-2024 в случае '
         'допуска россиян',
 'site': 'lenta.ru'}
{'_id':